<a href="https://colab.research.google.com/github/valterlucena/recuperacao-informacao/blob/master/evaluation/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
DATA_URL = 'https://raw.githubusercontent.com/Benardi/ri_lab_01/master/output/results.csv'
news = pd.read_csv(DATA_URL)
pd.options.display.max_colwidth = 160
news

,title,subtitle,author,date,section,text,url
0,“A sociedade foi Rubens Paiva não os facínoras que o mataram”,A decisão da juíza que proíbe as Forças Armadas de celebrarem golpe de 1964 contrapõe texto de militares ao de Ulysses Guimarães na Constituinte. Gilmar Men...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasília proibiu em caráter liminar nesta sexta-feira as Forças Armadas de comemorarem especialmente com uso de din...,https://brasil.elpais.com/brasil/2019/03/26/politica/1553638410_317117.html
1,Justiça suspende decisão que proibia Forças Armadas de celebrarem golpe de 1964,Liminar havia sido concedida na sexta-feira a noite mas menos de 24 horas depois foi derrubada,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Ivani Silva da Luz que completa 55 anos neste domingo a Justiça Federal derrubou a decisão. Por meio de uma l...,https://brasil.elpais.com/brasil/2019/03/30/politica/1553963400_195148.html
2,Governo Bolsonaro prega “negacionismo histórico” sobre a ditadura,Marcos Napolitano professor da USP diz que o discurso do Governo Bolsonaro sobre o golpe de 64 está mais para negacionismo pois “tem um ponto de partida ...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 uma estratégia polêmica de mudar a narrativa sobre como se conta a história trouxe à tona discussões antes iso...,https://brasil.elpais.com/brasil/2019/04/05/politica/1554419295_939718.html
3,Quando os pais de Gabo perceberam que tinham um filho mentiroso,Gustavo Tatis percorre o universo de García Márquez e seus segredos de família intimamente ligados à obra que a Netflix adaptará para as telas,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preocupavam que o menino piscasse tanto. Seu pai chegou a lhe pingar um colírio homeopático mas de pouco serviu. ...,https://brasil.elpais.com/brasil/2019/03/06/cultura/1551888378_718292.html
4,Rádios canadenses banem músicas de Michael Jackson da programação,Quebec Cogeco Media toma a decisão após queixas de ouvintes indignados com documentário sobre supostos abusos sexuais cometidos pelo cantor,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e sucessos mundiais de pararam de tocar em algumas rádios da província do Quebec (leste do ). Na verdade qual...,https://brasil.elpais.com/brasil/2019/03/06/cultura/1551902134_787786.html
5,Rosângela uma das crianças sequestradas por famílias de militares na ditadura,“Roubaram minha identidade” diz Rosângela Paraná cuja certidão de nascimento foi forjada pelo avô sargento.,Joana Oliveira,30/03/2019 22:21:08,Brasil,Rosângela Serra Paraná não sabe quantos anos tem ou onde nasceu. Sua certidão de nascimento registrada em 1967 no Catete (Rio de Janeiro) e datada em 1º de...,https://brasil.elpais.com/brasil/2019/03/29/politica/1553877780_122371.html
6,“Lógica de usar torturadores da ditadura no crime foi usada nas milícias”,Aloy Jupiara coautor de 'Os porões da contravenção' fala sobre o legado de profissionalização do crime deixado pelos anos de chumbo que perdura até hoje,Felipe Betim,01/04/2019 13:22:17,Brasil,Trajetória similar tiveram outros vários agentes públicos da ditadura militar brasileira (1964-1985). Como o capitão do Exército Aílton Guimarães Jorge o c...,https://brasil.elpais.com/brasil/2019/03/29/politica/1553885098_115676.html
7,Bolsonaro troca embaixada por escritório em Jerusalém mas não evita retaliação palestina,"Autoridade Palestina convoca embaixador no Brasil para consultas. Em visita ao Muro das Lamentações presidente diz que palestinos têm o “direito de reclamar""",Juan Carlos Sanz,01/04/2019 12:26:32,Brasil,A devoção dos cristãos evangélicos em seu apoio incondicional ao fica atrás das obrigações do para com os poderosos pecuaristas locais que exportam todos...,https://brasil.elpais.com/brasil/2019/03/31/politica/1554060705_325198.html
8,O lado mais sombrio de Dickens,Escritor e j

In [0]:
def isValid(token):
  return not bool(re.search(r'\d', token)) and len(token) > 2

def total_documents():
  return len(news)

def get_tokens(document):
  toker = RegexpTokenizer('''\w+[-']*\w*''')
  stop_words = stopwords.words('portuguese')
  return [token for token in toker.tokenize(document.lower()) if isValid(token) and token not in stop_words]

def build_index(documents):
  index = {}
  n = 0
  for document in documents:
    n += 1
    tokens = get_tokens(document)
    for token in tokens:
      occurrence = tokens.count(token)
      if token not in index:
        index[token] = {}
      if n not in index[token]:
        index[token][n] = occurrence
  return index

index = build_index(news.text)

for posting in index:
  k = len(index[posting])
  idf = round(np.log((total_documents() + 1) / k), 2)
  index[posting]['idf'] = idf
  
vocabulary = index.keys()

In [0]:
def binary_representation(query, document):
  '''
  Calcula a medida de similaridade entre a consulta e um documento
  pela representação binária do modelo vetorial
  '''
  terms = query.split()
  doc_tokens = get_tokens(document)
  q = {}
  d = {}
  for term in terms:
    q[term] = 0
    d[term] = 0
    if term in vocabulary:
      q[term] = 1
    if term in doc_tokens:
      d[term] = 1  
  measure = 0
  for term in terms:
    if q[term] != 0 and d[term] != 0:
      measure += q[term] * d[term]  
  return measure
  
def tf_representation(query, document):
  '''
  Calcula a medida de similaridade entre a consulta e um documento
  pela representação TF do modelo vetorial
  '''
  terms = query.split()
  doc_tokens = get_tokens(document)
  q = {}
  d = {}
  for term in terms:
    q[term] = 0
    d[term] = 0
    if term in vocabulary:
      q[term] = terms.count(term)
    if term in doc_tokens:
      d[term] = doc_tokens.count(term)
  measure = 0
  for term in terms:
    if q[term] != 0 and d[term] != 0:
      measure += q[term] * d[term]
  return measure

def tf_idf_representation(query, document):
  '''
  Calcula a medida de similaridade entre a consulta e um documento
  pela representação TF-IDF do modelo vetorial
  '''
  terms = query.split()
  doc_tokens = get_tokens(document)
  q = {}
  d = {}
  for term in terms:
    q[term] = 0
    d[term] = 0
    if term in vocabulary:
      q[term] = terms.count(term)
    if term in doc_tokens:
      d[term] = doc_tokens.count(term)
  measure = 0
  for term in terms:
    idf = index[term]['idf']
    if q[term] != 0 and d[term] != 0:
      measure += q[term] * d[term] * idf
  return round(measure, 2)

def bm25_representation(query, document, k):
  '''
    Calcula a medida de similaridade entre a consulta e um documento
    pela representação bm25 do modelo vetorial
  '''
  terms = query.split()
  doc_tokens = get_tokens(document)
  matched = [term for term in terms if term in doc_tokens]
  measure = 0
  for match in matched:
    cwq = terms.count(match)
    cwd = doc_tokens.count(match)
    m = total_documents()
    dfw = len(index[match].keys()) - 1
    measure += cwq * (((k + 1) * cwd) / (cwd  + k)) * np.log((m + 1) / dfw)
  return round(measure, 2)

In [0]:
def get_top_10(results):
  return sorted(results, key = lambda x: x[1], reverse=True)[:10]


def get_top_documents(query, documents):
  n = 0
  binary = []
  tf = []
  tf_idf = []
  bm25 = []
  for document in documents:
    binary.append((n,binary_representation(query, document)))
    tf.append((n, tf_representation(query, document)))
    tf_idf.append((n, tf_idf_representation(query, document)))
    bm25.append((n, bm25_representation(query, document, 10)))
    n += 1
  data = {
      'binary': get_top_10(binary),
      'tf': get_top_10(tf),
      'tf_idf': get_top_10(tf_idf),
      'bm25': get_top_10(bm25)
  }
  return data

In [0]:
query = "negacionismo histórico"

In [0]:
def reciprocal_rank(ranking):
  return 1 / ranking

def get_ranking(document, results):
  rank = 1
  for d,_ in results:
    if document == d:
      return rank
  return -1

In [35]:
data = pd.DataFrame(get_top_documents(query, news.text))
data

,binary,tf,tf_idf,bm25
0,"(2, 2)","(2, 5)","(2, 19.09)","(2, 16.48)"
1,"(9, 1)","(24, 2)","(247, 4.83)","(247, 4.83)"
2,"(16, 1)","(40, 2)","(24, 4.6)","(24, 4.22)"
3,"(18, 1)","(55, 2)","(40, 4.6)","(40, 4.22)"
4,"(21, 1)","(9, 1)","(55, 4.6)","(55, 4.22)"
5,"(24, 1)","(16, 1)","(9, 2.3)","(9, 2.3)"
6,"(40, 1)","(18, 1)","(16, 2.3)","(16, 2.3)"
7,"(55, 1)","(21, 1)","(18, 2.3)","(18, 2.3)"
8,"(56, 1)","(56, 1)","(21, 2.3)","(21, 2.3)"
9,"(81, 1)","(81, 1)","(56, 2.3)","(56, 2.3)"


In [38]:
ranking_binary = get_ranking(2, data['binary'])
reciprocal_rank(ranking_binary)

1.0

In [39]:
ranking_tf = get_ranking(2, data['tf'])
reciprocal_rank(ranking_tf)

1.0

In [44]:
ranking_tf_idf = get_ranking(2, data['tf_idf'])
reciprocal_rank(ranking_tf_idf)

1.0

In [45]:
ranking_bm25 = get_ranking(2, data['bm25'])
reciprocal_rank(ranking_bm25)

1.0

Pra todas as abordagens, o valor do reciprocal rank sempre foi 1, já que o documento escolhido, cujo índice é 2, sempre foi o primeiro colocado nos resultados da consulta.